In [182]:
import pandas as pd
from ast import literal_eval
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [183]:
df = pd.read_csv("../Data/Raw/movies_metadata.csv")
df.head(1)

C:\Users\MD Maruf\AppData\Local\Temp\ipykernel_27396\1899124903.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Data/Raw/movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [184]:
df.shape

(45466, 24)

In [185]:
df = df.drop(['adult', 'belongs_to_collection', 'budget', 'homepage', 'id', 'imdb_id', 'popularity', 'tagline', 'title', 'poster_path', 'production_companies', 'production_countries', 'original_language', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'video'], axis=1)
df.head()

,genres,original_title,overview,vote_average,vote_count
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,"Led by Woody, Andy's toys live happily in his ...",7.7,5415.0
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji,When siblings Judy and Peter discover an encha...,6.9,2413.0
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men,A family wedding reignites the ancient feud be...,6.5,92.0
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",6.1,34.0
4,"[{'id': 35, 'name': 'Comedy'}]",Father of the Bride Part II,Just when George Banks has recovered from his ...,5.7,173.0


In [186]:
df['overview'].isna().sum()

np.int64(954)

In [187]:
df['overview'] = df['overview'].fillna(" ")

In [188]:
print(f"Vote Count NAN value: {df['vote_count'].isna().sum()}")
print(f"Vote Average NAN value: {df['vote_average'].isna().sum()}")

Vote Count NAN value: 6
Vote Average NAN value: 6


In [189]:
df['vote_count'] = pd.to_numeric(df['vote_count'], errors='coerce')
df['vote_average'] = pd.to_numeric(df['vote_average'], errors='coerce')

df = df.dropna(subset=['vote_count'])
df = df.dropna(subset=['vote_average'])

In [190]:
df.loc[2345]['genres']

"[{'id': 28, 'name': 'Action'}, {'id': 35, 'name': 'Comedy'}, {'id': 80, 'name': 'Crime'}]"

In [191]:
df['original_title'].duplicated().sum()

np.int64(2091)

In [192]:
df = df.drop_duplicates(subset=['original_title'])

In [193]:
df.shape

(43369, 5)

In [194]:
df['genres'] = df['genres'].apply(literal_eval)
df['genres'] = df['genres'].apply(lambda x: " ".join([genre['name'] for genre in x]))

In [195]:
df[['original_title', 'genres']].head()

,original_title,genres
0,Toy Story,Animation Comedy Family
1,Jumanji,Adventure Fantasy Family
2,Grumpier Old Men,Romance Comedy
3,Waiting to Exhale,Comedy Drama Romance
4,Father of the Bride Part II,Comedy


In [196]:
df.loc[435][['original_title']]

original_title    Dave
Name: 435, dtype: object

In [197]:
df[['overview', 'genres']].isna().sum()

overview    0
genres      0
dtype: int64

In [198]:
df['description'] = df['overview'] + " " + df['genres']

In [199]:
df.loc[6754]['description']

'Composer Gordon Stuart brutaly rapes a fashion model, goes to trial, gets freed, comes back and rapes her little sister. She takes revenge. Drama Thriller'

In [200]:
df.loc[9875]['description']

'Ming dynasty noblewoman Yang must escape from the evil eunuch Hsu. She seeks refuge at a decrepit town where she gets assistance from a naive scholar & a group of mysterious yet powerful monks. Action Adventure'

In [201]:
noise_movies = df[(df['vote_count'] < 5) & (df['vote_average'] >= 8)]
df = df.drop(noise_movies.index)

In [202]:
def clean_text_column(text):
    text = text.lower()
    text = re.sub(r"\\'", "'", text)
    text = re.sub(r"[^\w\s']", " ", text)
    text = text.strip()
    text = re.sub(r"\s+", " ", text)

    words = text.split()
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if w not in stop_words]
    
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(w) for w in words]
    
    return " ".join(words)


In [203]:
# Apply the function to the column
df['description'] = df['description'].apply(clean_text_column)

In [204]:
df['original_title'] = df['original_title'].str.lower()

In [205]:
df = df.reset_index(drop=True)

In [206]:
(df['description'] == '').sum()

np.int64(237)

In [207]:
df = df[df['description'] != '']

In [208]:
df.head()

,genres,original_title,overview,vote_average,vote_count,description
0,Animation Comedy Family,toy story,"Led by Woody, Andy's toys live happily in his ...",7.7,5415.0,led woody andy's toy live happily room andy's ...
1,Adventure Fantasy Family,jumanji,When siblings Judy and Peter discover an encha...,6.9,2413.0,sibling judy peter discover enchanted board ga...
2,Romance Comedy,grumpier old men,A family wedding reignites the ancient feud be...,6.5,92.0,family wedding reignites ancient feud next doo...
3,Comedy Drama Romance,waiting to exhale,"Cheated on, mistreated and stepped on, the wom...",6.1,34.0,cheated mistreated stepped woman holding breat...
4,Comedy,father of the bride part ii,Just when George Banks has recovered from his ...,5.7,173.0,george bank recovered daughter's wedding recei...


In [209]:
df.shape

(41917, 6)

In [ ]:
df.to_csv(
    "../Data/processed/cleaned_dataset.csv",
    index=False
)